In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
dirs = ['ants', 'bees']

data = []
label = []

for i, d in enumerate(dirs):
    files = os.listdir('./data/hymenoptera_data/' + d)
    
    for f in files:
        img = Image.open('./data/hymenoptera_data/' + d + '/' + f, 'r')
        resize_img = img.resize((128,128))
        r,g,b = resize_img.split()
        r_resize_img = np.asarray(np.float32(r) / 255.)
        g_resize_img = np.asarray(np.float32(g) / 255.)
        b_resize_img = np.asarray(np.float32(b) / 255.)
        
        rgb_resize_img = np.asarray([r_resize_img, g_resize_img, b_resize_img])
        data.append(rgb_resize_img)
    
        label.append(i)

In [3]:
#ants --> 0
#bees --> 1
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [4]:
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')
print(data.shape)
print(label.shape)

(397, 3, 128, 128)
(397,)


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.1)

X_train = torch.from_numpy(X_train).float()
Y_train = torch.from_numpy(Y_train).long()

In [6]:
train = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train, batch_size=32, shuffle=True)

In [345]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(29 * 29 * 20, 100)
        self.fc2 = nn.Linear(100, 2)
        
        self.drop1 = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 29 * 29 * 20)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.fc2(x)
        
        return F.log_softmax(x)
    
model = Net()
model = model.cuda()
torch.backends.cudnn.benchmark=True

In [346]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002)

In [347]:
for epoch in range(100):
    total_loss = 0
    for x_train, y_train in train_loader:
        x_train , y_train = Variable(x_train).cuda(), Variable(y_train).cuda()
        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    if (epoch + 1) % 10 == 0:
        print(epoch + 1, total_loss)

C:\Users\my\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


10 6.817331075668335
20 4.886314779520035
30 3.1401433646678925
40 1.8646582439541817
50 0.8642376773059368
60 0.4906966220587492
70 0.39313986990600824
80 0.33934136014431715
90 0.12882581679150462
100 0.1126225006300956


In [352]:
model.eval()

Net(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16820, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=2, bias=True)
  (drop1): Dropout(p=0.5)
)

In [353]:
X_test = np.array(X_test, dtype='float32')
Y_test = np.array(Y_test, dtype='int64')

X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(Y_test).long()

print(X_test.shape)
print(Y_test.shape)

torch.Size([40, 3, 128, 128])
torch.Size([40])


In [354]:
x_test, y_test = Variable(X_test).cuda(), Variable(Y_test).cuda()
result = torch.max(model(x_test).data, 1)[1]
accuracy = sum(y_test.cpu().numpy() == result.cpu().numpy()) / len(y_test.cpu().numpy())

accuracy

C:\Users\my\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.7